### 모듈 임포트

In [1]:
import os
import json, sys, re
import shutil
import collections
import glob
import re
import json
import hashlib, itertools
import csv   
import pandas as pd

from pathlib import Path
from typing   import Union, Dict, List, Tuple
from tqdm.auto import tqdm
from pathlib import Path
from tqdm.auto import tqdm
from collections import defaultdict

### unpacking

In [2]:
tasks = [
    # (source, target)
    ('./Origin/Sublabel',                 './converted/Sublabel'),
    ('./Origin/Training/02.라벨링데이터',   './converted/Training'),
    ('./Origin/Validation/02.라벨링데이터', './converted/Validation'),
]

for source_dir, target_dir in tasks:
    copied_files = 0

    if not os.path.isdir(source_dir):
        print(f"[❌ 오류] 경로 '{source_dir}'가 없음")
        continue

    os.makedirs(target_dir, exist_ok=True)
    print(f"[🚀] {source_dir} 하위의 모든 .json을 {target_dir}로 평탄화(flatten)")

    for root, dirs, files in os.walk(source_dir):
        if root == target_dir:
            continue
        for file_name in files:
            if not file_name.endswith('.json'):
                continue

            src = os.path.join(root, file_name)
            dst = os.path.join(target_dir, file_name)

            # ── 파일명 충돌 대비 ─────────────────────────
            if os.path.exists(dst):
                base, ext = os.path.splitext(file_name)
                i = 1
                while os.path.exists(os.path.join(target_dir, f"{base}_{i}{ext}")):
                    i += 1
                new_name = f"{base}_{i}{ext}"
                dst = os.path.join(target_dir, new_name)
                print(f"⚠️ 파일명 충돌: '{file_name}' → '{new_name}' 로 복사")

            try:
                shutil.copy2(src, dst)    
                copied_files += 1
            except Exception as e:
                print(f"[오류] '{src}' 복사 실패: {e}")

    print(f"[✅ 완료] {target_dir} 에 {copied_files}개 복사 완료\n")

[🚀] ./Origin/Sublabel 하위의 모든 .json을 ./converted/Sublabel로 평탄화(flatten)
[✅ 완료] ./converted/Sublabel 에 2345개 복사 완료

[🚀] ./Origin/Training/02.라벨링데이터 하위의 모든 .json을 ./converted/Training로 평탄화(flatten)
[✅ 완료] ./converted/Training 에 1446개 복사 완료

[🚀] ./Origin/Validation/02.라벨링데이터 하위의 모든 .json을 ./converted/Validation로 평탄화(flatten)
[✅ 완료] ./converted/Validation 에 286개 복사 완료



### 데이터셋 확인

In [3]:
isbns = set()
for dirpath, _, filenames in os.walk('./Origin/Training'):
    for fn in filenames:
        if fn.lower().endswith('.json'):
            with open(os.path.join(dirpath, fn), encoding='utf-8-sig') as f:
                d = json.load(f)
            books = d if isinstance(d, list) else [d]
            for book in books:
                isbns.add(str(book.get('isbn','')).strip())
print(f"원본 폴더 기준 훈련 데이터 고유권수 : {len(isbns)}")  # 원본 폴더 기준 고유권수

원본 폴더 기준 훈련 데이터 고유권수 : 1446


In [4]:
isbns = set()
for dirpath, _, filenames in os.walk('./Origin/Validation'):
    for fn in filenames:
        if fn.lower().endswith('.json'):
            with open(os.path.join(dirpath, fn), encoding='utf-8-sig') as f:
                d = json.load(f)
            books = d if isinstance(d, list) else [d]
            for book in books:
                isbns.add(str(book.get('isbn','')).strip())
print(f"원본 폴더 기준 검증 데이터 고유권수 : {len(isbns)}")  # 원본 폴더 기준 고유권수

원본 폴더 기준 검증 데이터 고유권수 : 286


### 명시적 질문만 남기기

★ 라벨링 데이터만 사용

In [5]:
# ╔═══════════════════════════════════════╗
# ║   KorQuAD v1 말뭉치 + cleaned 본문   ║
# ╚═══════════════════════════════════════╝
# ───────────────── 경로 ─────────────────
ROOT = Path("./converted") ; ROOT.mkdir(exist_ok=True)     
CLEAN_DIR = ROOT / "cleaned"       ; CLEAN_DIR.mkdir(exist_ok=True)
FMT_DIR   = ROOT / "formatted"     ; FMT_DIR.mkdir(exist_ok=True)

LBL_DIRS  = {"training": ROOT / "training",
             "validation": ROOT / "validation"}

# ───────────── ISBN 추출 util ────────────
ISBN_RE = re.compile(r"([0-9X]{9,13})(?=_?\.json$)", re.I)
def isbn_from(path: Union[str, Path]) -> str:
    path = Path(path)
    m = ISBN_RE.search(path.name)
    if not m:
        raise ValueError(f"ISBN not found in filename: {path.name}")
    return m.group(1)

# ───────────── 정규화 & back-map ──────────
PUNCT = re.compile(r"[^\w\s]")      # 필요하면 범위 조정
def normalize_with_map(text: str) -> Tuple[str, List[int]]:
    norm_chars, back = [], []
    for i, ch in enumerate(text):
        if PUNCT.match(ch):           # 문장부호 skip
            continue
        norm_chars.append(ch)
        back.append(i)
    return "".join(norm_chars), back

def locate_answer(ctx: str, ans: str) -> Tuple[int, str] | Tuple[None, None]:
    """원문 ctx 에서 ans 위치 반환. 실패하면 punctuation-agnostic 재탐색."""
    pos = ctx.find(ans)
    if pos != -1:
        return pos, ans
    # ― 재탐색 ―
    ctx_norm, back = normalize_with_map(ctx)
    ans_norm, _    = normalize_with_map(ans)
    p2 = ctx_norm.find(ans_norm)
    if p2 == -1:
        return None, None
    start = back[p2]
    end   = back[p2 + len(ans_norm) - 1]
    return start, ctx[start:end+1]

# ───────────── Split 처리 함수 ────────────
def process_split(split: str, src_root: Path) -> None:
    korquad_data   : List[dict]      = []
    total_qas = missed_qas = 0
    missed_log : list[tuple] = []    # (isbn, Q, A)

    for lbl_file in tqdm(list(src_root.glob("*.json")), desc=split.upper()):
        isbn = isbn_from(lbl_file)
        book  = json.loads(lbl_file.read_text(encoding="utf-8-sig"))

        # ①  context  ─ 페이지 순서대로 모아 붙이기
        paras = sorted(book.get("paragraphInfo", []),
                       key=lambda p: p.get("srcPage", 0))
        context = "\n\n".join(p.get("srcText", "") for p in paras).strip()

        # cleaned/<isbn>.json : 한 번만 저장
        cleaned_path = CLEAN_DIR / f"{isbn}.json"
        if not cleaned_path.exists():
            cleaned_path.write_text(
                json.dumps({"isbn": isbn,
                            "title": book.get("title", ""),
                            "text": context},
                           ensure_ascii=False, indent=2),
                encoding="utf-8-sig"
            )

        # ②  KorQuAD QA 변환  (명시적 질문만)
        qas: List[dict] = []
        for qa in itertools.chain.from_iterable(
                p.get("queAnsPairInfo", []) for p in paras):
            if "명시적" not in qa.get("ansType", ""):
                continue
            total_qas += 1
            qid = f"{isbn}-{hashlib.md5(qa['question'].encode()).hexdigest()[:8]}"
            ans_raw = qa.get("ansM1", "").strip()
            if not ans_raw:
                missed_qas += 1
                missed_log.append((isbn, qa["question"], "<EMPTY>"))
                continue
            start, span = locate_answer(context, ans_raw)
            if start is None:
                missed_qas += 1
                missed_log.append((isbn, qa["question"], ans_raw))
                continue
            qas.append({
                "id": qid,
                "question": qa["question"],
                "answers": [{"text": span, "answer_start": start}]
            })

        if qas:          # QA 가 하나라도 있어야 포함
            korquad_data.append({
                "title": book.get("title", isbn),
                "paragraphs": [{
                    "context": context,
                    "qas": qas
                }]
            })

    # ③  KorQuAD 파일 저장
    out_file = FMT_DIR / f"{split}.json"
    out_file.write_text(
        json.dumps({"version": "v1.0", "data": korquad_data},
                   ensure_ascii=False, indent=2),
        encoding="utf-8"
    )

    # ④  리포트
    print(f"\n📜 {out_file.name}")
    print(f"  articles   : {len(korquad_data):5d}")
    print(f"  paragraphs : {len(korquad_data):5d}")
    print(f"  QAs        : {total_qas:5d}")
    print(f"    └─ 매칭 실패 : {missed_qas}")
    if missed_log:
        print("  ── mismatch preview ──")
        for isbn, q, a in missed_log[:5]:
            print(f"   • [{isbn}] {q}  →  {a}")

# ────────────────────── main ──────────────────────
if __name__ == "__main__":
    for split, root in LBL_DIRS.items():
        process_split(split, root)

    print("\n🟢 All done! → cleaned/, formatted/  디렉터리 확인")


TRAINING:   0%|          | 0/1446 [00:00<?, ?it/s]


📜 training.json
  articles   :  1442
  paragraphs :  1442
  QAs        : 47443
    └─ 매칭 실패 : 1159
  ── mismatch preview ──
   • [9791128216893] ㅂ으로 시작되는 이름을 가진 과일 중 길쭉한 것은 무엇일까요?  →  바나나
   • [9791128216893] 첫 글자를 보고 우리는 무엇을 해야 하나요?  →  첫 글자로 시작하는 이름을 찾아야 해요.
   • [9791159424502] 아이들은 정원에서 어떻게 놀았나요?  →  신나게 놀았어요.
   • [9791159424564] 꼬투리를 꺾었을 때 완두콩 오형제는 바깥세상으로 나갈 생각에 어떻게 했나요?  →  완두콩 오형제는 가슴이 두근거렸어요.
   • [9791159424632] 빨간 모자는 어디를 걸어갔나요?  →  숲속


VALIDATION:   0%|          | 0/286 [00:00<?, ?it/s]


📜 validation.json
  articles   :   282
  paragraphs :   282
  QAs        :  5998
    └─ 매칭 실패 : 174
  ── mismatch preview ──
   • [9791128216954] 빗방울이 창문을 두드린 이유는 무엇인가요?  →  집에만 있지말고 나와서 놀자고
   • [9791159424700] 이상한 일은 어디에서 일어났나요?  →  할머니 집
   • [9791165434724] 아이의 배변을 도와주려고 하는 이는 누구인가요?  →  야옹이
   • [9791165435950] 책은 혼자 보는 것보다 어떻게 보는 것이 더 재미있을까요?  →  친구와 함께 보면 더 재미나요.
   • [9791165436025] 엄마 배에 손을 대어 본 토리는 아기가 뭘 한다고 생각했나요?  →  깡총깡총 뛰나 봐요.

🟢 All done! → cleaned/, formatted/  디렉터리 확인


### formatting

In [6]:
# ╔══════════════════════════════════════════════════════╗
# ║  🟢  Cell 1   ║  KorQuAD 생성 + 매칭 실패 디버그 로그 ║
# ╚══════════════════════════════════════════════════════╝

ROOT          = Path("./converted")
TRAIN_LB      = ROOT / "training"
VAL_LB        = ROOT / "validation"
CLEAN_DIR     = ROOT / "cleaned"     
OUT_DIR       = ROOT / "formatted"   

ISBN_RE = re.compile(r"(\d{9,13}X?)_?(?=\.json$)")
def isbn_from_name(p: Union[str, Path]) -> str:
    m = ISBN_RE.search(Path(p).name)
    if not m: raise ValueError(f"ISBN not found in {p}")
    return m.group(1)

def load_pool(dir_: Path) -> Dict[str, dict]:
    return {isbn_from_name(f): json.loads(f.read_text("utf-8-sig"))
            for f in dir_.glob("*.json")}

# ───────── cleaned 파일 복사 ─────────
def save_clean(isbn: str, book: dict):
    pages = sorted(book["paragraphInfo"], key=lambda x: x.get("srcPage", 0))
    (CLEAN_DIR / f"{isbn}.json").write_text(
        json.dumps(
            {"isbn": isbn,
             "title": book.get("title", ""),
             "pages":[{"page":p.get("srcPage"),"text":p["srcText"]} for p in pages]},
            ensure_ascii=False, indent=2),
        encoding="utf-8"
    )

# ───────── 페이지→KorQuAD + miss 수집 ─────────
def para_to_korquad(isbn: str, title: str, para: dict
) -> Tuple[dict, List[dict]]:
    ctx = para["srcText"]
    page = para.get("srcPage")
    qas, misses = [], []
    for qa in para.get("queAnsPairInfo", []):
        ans = qa.get("ansM1")
        if not ans:
            continue
        start = ctx.find(ans)
        if start == -1:
            near = ctx[:300] if len(ctx) < 300 else ctx[max(0,start-15):start+len(ans)+15]
            misses.append(
                {"isbn":isbn, "page":page, "question":qa["question"], "answer":ans,
                 "ctx_slice":near.replace("\n"," "), "reason":"not-found"}
            )
            continue
        qid = f"{isbn}-{hashlib.md5(qa['question'].encode()).hexdigest()[:8]}"
        qas.append({"id":qid,"question":qa["question"],
                    "answers":[{"text":ans,"answer_start":start}]})
    return ({"title":title,"paragraphs":[{"context":ctx,"qas":qas}]}, misses)

# ───────── split 처리 ─────────
def split_to_json(pool: Dict[str, dict], split_name: str):
    korquad, miss_log = [], []
    total_q, miss_q = 0, 0

    for isbn, book in tqdm(pool.items(), desc=split_name.upper()):
        save_clean(isbn, book)

        for para in sorted(book["paragraphInfo"],
                           key=lambda x: x.get("srcPage",0)):
            total_q += len(para.get("queAnsPairInfo",[]))
            rec, miss = para_to_korquad(isbn, book.get("title",""), para)
            miss_q   += len(miss)
            miss_log.extend(miss)
            if rec["paragraphs"][0]["qas"]:
                korquad.append(rec)

    # KorQuAD json
    out_json = OUT_DIR / f"{split_name}.json"
    json.dump({"version":"v1.0","data":korquad},
              out_json.open("w",encoding="utf-8"),
              ensure_ascii=False, indent=2)

    # miss CSV
    if miss_log:
        miss_csv = OUT_DIR / f"miss_{split_name}.csv"
        with miss_csv.open("w",newline="",encoding="utf-8-sig") as f:
            writer = csv.DictWriter(
                f, fieldnames=["split","isbn","page","question","answer","ctx_slice","reason"]
            )
            writer.writeheader()
            for row in miss_log:
                row["split"]=split_name
                writer.writerow(row)

    # 요약 출력
    print(f"\n✅ {out_json.name:14}  paragraphs:{len(korquad):7d}")
    print(f"   total QA: {total_q:6d} | missed: {miss_q:5d}")
    if miss_log:
        print("   ↪️  miss example:")
        for ex in miss_log[:3]:
            print(f"      • ISBN {ex['isbn']} p{ex['page']} Q:{ex['question']}  A:{ex['answer']}")

# ───────── main ─────────
train_pool, val_pool = load_pool(TRAIN_LB), load_pool(VAL_LB)
split_to_json(train_pool, "training")
split_to_json(val_pool,   "validation")
print("\n🟢  Done — check formatted/miss_*.csv for full mismatch list.")


TRAINING:   0%|          | 0/1446 [00:00<?, ?it/s]


✅ training.json   paragraphs:  27662
   total QA:  80043 | missed: 33615
   ↪️  miss example:
      • ISBN 9788961915915 p4 Q:어느 날 할머니가 꽃밭에 물을 주고 있는데 무슨 일이 벌어졌나요?  A:갑자기 하늘에서 커다란 우박이 떨어졌어요.
      • ISBN 9788961915953 p4 Q:신의가 신이 나서 힘껏 달려간 이유는 무엇인가요?  A:아빠와 모처럼 나들이를 나왔기 때문이에요.
      • ISBN 9788961916080 p5 Q:같은 공룡을 잡아먹는 육식 공룡은 어때 보여요?  A:무시무시해 보여요.


VALIDATION:   0%|          | 0/286 [00:00<?, ?it/s]


✅ validation.json  paragraphs:   3249
   total QA:  10005 | missed:  4179
   ↪️  miss example:
      • ISBN 9788961916028 p4 Q:내가 주는 힌트를 듣고 무엇을 해 볼 것이냐고 물어봤나요?  A:내가 누군지 맞혀 볼 것이냐고 물어봤어요.
      • ISBN 9788961916103 p3 Q:릴리엔탈과 구스타프는 하늘의 새를 구경하기 위해 매일 무엇을 했나요?  A:언덕에 올랐어요.
      • ISBN 9788961916103 p3 Q:형제는 자유롭게 하늘을 나는 새를 보며 어떤 감정이 들었나요?  A:새를 부러워했어요.

🟢  Done — check formatted/miss_*.csv for full mismatch list.


### validate

In [7]:
# ╔════════════════════════════════════════════╗
# ║  🟢  Cell 2   ║  KorQuAD / cleaned 검증    ║
# ╚════════════════════════════════════════════╝
import json, csv, re
from pathlib import Path
from typing import List, Dict
from tqdm.auto import tqdm

ROOT       = Path("./converted")
FMT_DIR    = ROOT / "formatted"
CLEAN_DIR  = ROOT / "cleaned"

SPLITS = {"training":"training.json", "validation":"validation.json"}

def verify_korquad(split:str, file:Path)-> List[Dict]:
    """answer_start 정합성 & 범위 체크 → 실패 row 목록 반환"""
    bad : List[Dict] = []
    data = json.loads(file.read_text(encoding="utf-8"))
    for art in data["data"]:
        title  = art.get("title","")
        for para in art["paragraphs"]:
            ctx    = para["context"]
            ctxlen = len(ctx)
            for qa in para["qas"]:
                for ans in qa["answers"]:
                    text  = ans["text"]
                    start = ans["answer_start"]
                    # ① 답이 context에 존재?
                    found_at = ctx.find(text)
                    # ② start 범위, slice 일치
                    slice_ok = 0 <= start <= ctxlen-len(text) and ctx[start:start+len(text)]==text
                    if slice_ok and found_at==start:
                        continue   # 정상
                    # --- 문제 기록
                    reason  = ("not-found"      if found_at==-1 else
                               "misalign"      if found_at!=start else
                               "slice-mismatch")
                    bad.append({
                        "split":split,
                        "title":title,
                        "id":qa["id"],
                        "question":qa["question"],
                        "answer":text,
                        "start":start,
                        "found_at":found_at,
                        "reason":reason,
                        "ctx_slice":ctx[max(0,start-20):min(ctxlen,start+len(text)+20)].replace("\n"," ")
                    })
    return bad

def save_csv(rows:List[Dict], out:Path):
    if not rows: return
    with out.open("w",newline="",encoding="utf-8-sig") as f:
        w=csv.DictWriter(f,fieldnames=list(rows[0]))
        w.writeheader(); w.writerows(rows)

# ───── 1) KorQuAD 파일 검사 ─────
isbn_in_kqa=set()
for split,fname in SPLITS.items():
    fpath = FMT_DIR / fname
    if not fpath.exists(): 
        print(f"⚠️  {fname} not found"); continue
    bad = verify_korquad(split,fpath)
    save_csv(bad, FMT_DIR/f"bad_{split}.csv")
    # 통계
    total   = sum( len(p["qas"]) for a in json.loads(fpath.read_text(encoding="utf-8"))["data"]
                                for p in a["paragraphs"])
    print(f"\n🔍 {fname}")
    print(f"  QA total : {total:,}")
    print(f"  bad rows : {len(bad):,}   ⇢  saved to bad_{split}.csv" if bad else "  ✅ all offsets OK")
    if bad:                        # 화면 예시
        for row in bad[:3]:
            print(f"   • {row['id']} | reason:{row['reason']} "
                  f" start {row['start']} / found {row['found_at']} : “{row['answer']}”")

    # ISBN 추적
    for art in json.loads(fpath.read_text(encoding="utf-8"))["data"]:
        tit = art.get("title","")
        m = re.fullmatch(r"\d{9,13}X?", tit)
        if m: isbn_in_kqa.add(m.group())

# ───── 2) cleaned 파일 존재 체크 ─────
missing = [ {"isbn":isbn} for isbn in sorted(isbn_in_kqa)
            if not (CLEAN_DIR/f"{isbn}.json").exists() ]
save_csv(missing, FMT_DIR/"missing_cleaned.csv")

print(f"\n📂 cleaned dir : {len(list(CLEAN_DIR.glob('*.json'))):,} files")
print(f"   KorQuAD ISBN referenced : {len(isbn_in_kqa):,}")
print(f"   missing cleaned files   : {len(missing):,}"
      + (f"  ⇢ saved to missing_cleaned.csv" if missing else " (none)"))

print("\n🟢  verification done")



🔍 training.json
  QA total : 46,428
  ✅ all offsets OK

🔍 validation.json
  QA total : 5,826
  ✅ all offsets OK

📂 cleaned dir : 1,732 files
   KorQuAD ISBN referenced : 0
   missing cleaned files   : 0 (none)

🟢  verification done


### Metatdata 생성 (for web)

In [8]:
# ╔══════════════════════════════╗
# ║  🟢  Cell 3 – meta 파일 2종   ║
# ╚══════════════════════════════╝

ROOT      = Path("./converted")
TRAIN_LB  = ROOT / "training"
VAL_LB    = ROOT / "validation"
META_DIR   = ROOT / "meta"; META_DIR.mkdir(exist_ok=True)
ISBN_RE = re.compile(r"([0-9X]{9,13})(?=_?\.json$)")

def isbn_from_name(p:Path)->str:
    m = ISBN_RE.search(p.name)
    return m.group(1) if m else None

def load_pool(p_dir:Path):
    return {isbn_from_name(f):json.loads(f.read_text("utf-8-sig"))
            for f in p_dir.glob("*.json")}

def to_meta_dict(isbn:str, book:dict):
    meta = {
        "isbn"         : isbn,
        "title"        : book.get("title","").strip(),
        "author"       : book.get("author"),
        "illustrator"  : book.get("illustrator"),
        "publisher"    : book.get("publisher"),
        "publishedYear": book.get("publishedYear"),
        "readAge"      : book.get("readAge"),
        "classification": book.get("classification"),
    }
    # 빈 값은 제거
    return {k:v for k,v in meta.items() if v}

def build_meta(pool:dict, split:str):
    metas = [to_meta_dict(i,bk) for i,bk in pool.items()]
    out   = META_DIR / f"book_meta_all_{split}.json"
    out.write_text(json.dumps(metas, ensure_ascii=False, indent=2),
                   encoding="utf-8")
    print(f"💾  {out.relative_to(ROOT)}  ({len(metas)} books)")

train_pool = load_pool(TRAIN_LB)
val_pool   = load_pool(VAL_LB)

build_meta(train_pool, "training")
build_meta(val_pool,   "validation")
print("🟢  meta files created")


💾  meta\book_meta_all_training.json  (1446 books)
💾  meta\book_meta_all_validation.json  (286 books)
🟢  meta files created


### Merge

In [9]:
# 파일 경로
TRAIN_META = Path('./converted/meta/book_meta_all_training.json')
VAL_META   = Path('./converted/meta/book_meta_all_validation.json')
OUT_META   = Path('./converted/meta/book_meta_all.json')

# 로드
with TRAIN_META.open(encoding='utf-8-sig') as f:
    train_meta = json.load(f)
with VAL_META.open(encoding='utf-8-sig') as f:
    val_meta = json.load(f)

# ISBN 중복 제거 (training 우선, 없으면 validation)
meta_dict = {item['isbn']: item for item in val_meta}
meta_dict.update({item['isbn']: item for item in train_meta})  # train이 우선

# 저장
with OUT_META.open('w', encoding='utf-8-sig') as f:
    json.dump(list(meta_dict.values()), f, ensure_ascii=False, indent=2)

print(f"✅ {OUT_META} 저장 완료 ({len(meta_dict)} 권)")


✅ converted\meta\book_meta_all.json 저장 완료 (1732 권)


# 끝 !